In [1]:
import os
os.chdir("../")


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [3]:

from spellX.constants import *
from spellX.utils.common import read_yaml , create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [5]:
import os
import urllib.request as request
import zipfile
from spellX import logger
from spellX.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [13]:
import traceback
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    print(f"An error occurred: {e}")
    # You can also print the traceback for more information

    traceback.print_exc()


[2023-08-29 18:29:41,335: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-29 18:29:41,340: INFO: common: yaml file: params.yaml loaded successfully]
An error occurred: yaml file is empty


Traceback (most recent call last):
  File "c:\users\p52s\documents\python project\spellx\src\spellX\utils\common.py", line 31, in read_yaml
    return ConfigBox(content)
  File "box\box.py", line 259, in box.box.Box.__init__
box.exceptions.BoxValueError: First argument must be mapping or iterable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\P52s\AppData\Local\Temp\ipykernel_6352\718821345.py", line 3, in <module>
    config = ConfigurationManager()
  File "C:\Users\P52s\AppData\Local\Temp\ipykernel_6352\221262974.py", line 8, in __init__
    self.params = read_yaml(params_filepath)
  File "c:\Users\P52s\.conda\envs\cnn\lib\site-packages\ensure\main.py", line 849, in __call__
    return_val = self.f(*args, **kwargs)
  File "c:\users\p52s\documents\python project\spellx\src\spellX\utils\common.py", line 33, in read_yaml
    raise ValueError("yaml file is empty")
ValueError: yaml file is empty
